<a href="https://colab.research.google.com/github/gihoonpark/-Korean_News_summarization-NLP/blob/main/%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%9A%94%EC%95%BD_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data load

In [ ]:
import json

with open('/content/drive/My Drive/dataset/생성요약/train.jsonl', 'r') as json_file:
    json_list = list(json_file)
with open('/content/drive/MyDrive/dataset/생성요약/abstractive_test_v2.jsonl', 'r') as json_file2:
    json_list2 = list(json_file2)

In [ ]:
text_list_train = []
summary_list_train = []

for json_str in json_list:
    result = json.loads(json_str)
    text_list_train.append(result.get('article_original'))
    summary_list_train.append(result.get('abstractive'))

txt_train = pd.Series(text_list_train)
summary_train = pd.Series(summary_list_train)

txt_col = []
for i in range(len(txt_train)):
    txt_col.append(" ".join(txt_train[i]))
txt_col = pd.Series(txt_col)
train_df = pd.concat([txt_col, summary_train], axis=1)

Data cleaning

In [ ]:
train_df[0] = train_df[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_df[1] = train_df[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_df.replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_df = train_df.dropna(how='any') # Null 값 제거

In [ ]:
train_df[0] = train_df[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") # 정규 표현식 수행
train_df[1] = train_df[1].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") # 정규 표현식 수행
train_df.replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_df = train_df.dropna(how='any') # Null 값 제거

In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '만', '게임', '겜', '되', '음', '면']

In [ ]:
text_list_test = []

for json_str in json_list2:
    result2 = json.loads(json_str)
    text_list_test.append(result2.get('article_original'))

txt_test = pd.Series(text_list_test)
txt_col2 = []
for i in range(len(txt_test)):
    txt_col2.append(" ".join(txt_test[i]))
test_df = pd.Series(txt_col2)
summary_test = pd.Series([])
test_df = pd.concat([test_df, summary_test], axis=1)

test_df[0] = test_df[0].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9 ]","") # 정규 표현식 수행

Data tokenizing

In [ ]:
mecab = Mecab()
train_df['X_tokenized'] = train_df[0].apply(mecab.morphs)
train_df['y_tokenized'] = train_df[1].apply(mecab.morphs)
train_df['X_tokenized'] = train_df['X_tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
train_df['y_tokenized'] = train_df['y_tokenized'].apply(lambda x: [item for item in x if item not in stopwords])
test_df['X_tokenized'] = test_df[0].apply(mecab.morphs)
test_df['X_tokenized'] = test_df['X_tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
# 보류
X_train = train_df['X_tokenized'].values
y_train = train_df['y_tokenized'].values
X_test= test_df['X_tokenized'].values

In [ ]:
# 요약 데이터에는 시작 토큰과 종료 토큰을 추가
train_df['decoder_input'] = train_df['y_tokenized'].apply(lambda x : ['sostoken']+ x)
train_df['decoder_target'] = train_df['y_tokenized'].apply(lambda x : x + ['eostoken'])

In [ ]:
encoder_input = np.array(train_df['X_tokenized'])
decoder_input = np.array(train_df['decoder_input'])
decoder_target = np.array(train_df['decoder_target'])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(encoder_input)

In [ ]:
'''
threshold = 5
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
'''

In [ ]:
src_vocab = 42000
src_tokenizer = Tokenizer(num_words = src_vocab) 
src_tokenizer.fit_on_texts(encoder_input)

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input) 
encoder_input_test = src_tokenizer.texts_to_sequences(X_test)

In [ ]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input)

In [ ]:
'''
threshold = 6
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)
'''

In [ ]:
tar_vocab = 15000
tar_tokenizer = Tokenizer(num_words = tar_vocab) 
tar_tokenizer.fit_on_texts(decoder_input)
tar_tokenizer.fit_on_texts(decoder_target)

In [ ]:
# model에 들어갈 최종 dataset
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input) 
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input)

Data padding

In [ ]:
text_max_len = 510
summary_max_len = 75
hidden_size = 64

encoder_input_train = pad_sequences(encoder_input_train, maxlen = text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen = summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen = summary_max_len, padding='post')

encoder_input_test = pad_sequences(encoder_input_test, maxlen = text_max_len, padding='post')


In [ ]:
drop_train = [index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1]
drop_test = [index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1]
encoder_input_train = np.delete(encoder_input_train, drop_train, axis=0)
decoder_input_train = np.delete(decoder_input_train, drop_train, axis=0)
decoder_target_train = np.delete(decoder_target_train, drop_train, axis=0)

encoder_input_test = np.delete(encoder_input_test, drop_test, axis=0)
decoder_input_test = np.delete(decoder_input_test, drop_test, axis=0)

print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :',len(decoder_input_train))
print('테스트 데이터의 개수 :',len(encoder_input_test))

Model & Training - Bidirectional LSTM, Attention mechanism

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [ ]:
checkpoint_path = "/content/drive/MyDrive/dataset/생성요약/training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# 인코더
encoder_inputs = Input(shape=(text_max_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, 256)(encoder_inputs)

# 인코더의 LSTM 1
encoder_lstm1 = LSTM(64, return_sequences=True, return_state=True ,dropout = 0.2, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
encoder_lstm3 = LSTM(64, return_state=True, return_sequences=True, dropout=0.2, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# 디코더
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, 200)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
decoder_lstm = LSTM(64, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation = 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs) 

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()


In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/src/layers/attention.py", filename="attention.py")
from attention import AttentionLayer

In [ ]:
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train,
          batch_size = 256, callbacks=[es, cp_callback], epochs = 40)

Re-train

In [ ]:
checkpoint_path = "/content/drive/MyDrive/dataset/생성요약/training_6/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
model.load_weights(checkpoint_path) # 가중치 불러오기

In [ ]:
checkpoint_path2 = "/content/drive/MyDrive/dataset/생성요약/training_6/cp.ckpt"
checkpoint_dir2 = os.path.dirname(checkpoint_path2)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path2,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train,
          batch_size = 256, callbacks=[es, cp_callback2], epochs = 30)

**Test**

In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])

In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(64,))
decoder_state_input_c = Input(shape=(64,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM이 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat) 

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['sostoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if(sampled_token!='eostoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'eostoken'  or len(decoded_sentence.split()) >= (summary_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
from tqdm import tqdm
sub_df = []
for i in tqdm(range(len(encoder_input_test))):
    sub_df.append(decode_sequence(encoder_input_test[i].reshape(1, text_max_len)))


In [ ]:
sub_df

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/dataset/생성요약/abstractive_sample_submission_v2.csv', index_col=0)
sub['summary'] = sub_df
sub.head()

In [ ]:
sub.to_csv('/content/drive/MyDrive/dataset/생성요약/abstractive_sample_submission_v2.csv')